In [1]:
#!pip install optuna

31/12/2022

In [2]:
import optuna
import xgboost as xgb
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('Admission_Predict.csv')
df 

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [4]:
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [5]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,115.614301,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,100.750000,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,200.500000,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,300.250000,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,400.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


In [6]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [7]:
x = df.drop(columns=['Serial No.','Chance of Admit '])
x 

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
395,324,110,3,3.5,3.5,9.04,1
396,325,107,3,3.0,3.5,9.11,1
397,330,116,4,5.0,4.5,9.45,1
398,312,103,3,3.5,4.0,8.78,0


In [8]:
y = df['Chance of Admit ']
y 

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
395    0.82
396    0.84
397    0.91
398    0.67
399    0.95
Name: Chance of Admit , Length: 400, dtype: float64

In [9]:
from sklearn.preprocessing import StandardScaler
std_sca = StandardScaler()
x = std_sca.fit_transform(x)

In [10]:
from sklearn.metrics import mean_squared_error
def objective(trail, data = x,target=y):
    x_train,x_test,y_train,y_test = train_test_split(data,target,test_size=.10,random_state=30)
    param={
        'tree_method':'hist',
        'lambda':trail.suggest_loguniform('lambda',1e-4,10.0),
        'alpha':trail.suggest_loguniform('alpha',1e-4,10.0),
        'colsample_bytree':trail.suggest_categorical('colsample_bytree',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample':trail.suggest_categorical('subsample',[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate':trail.suggest_categorical('learning_rate',[.00001,.0003,.008,.02,0.1,10,20]),
        'n_estimator':30000,
        'max_depth':trail.suggest_categorical('max_depth',[3,4,5,6,7,8,9,10,11,12]),
        'random_state':trail.suggest_categorical('random_state',[10,20,30,2000,3454,243123]),
        'mim_child_weight':trail.suggest_int('min_child_weight',1,200)
    }
    xgb_reg_model = xgb.XGBRegressor(**param)
    xgb_reg_model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True)
    pred_xgb = xgb_reg_model.predict(x_test)
    rmse = mean_squared_error(y_test,pred_xgb)
    return rmse 

In [11]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials = 10)
find_param.best_trial.params 

[I 2022-12-31 19:32:31,556] A new study created in memory with name: no-name-aef552f3-4653-495e-9049-49c3f6a026d8


[19:32:31] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "mim_child_weight", "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:0.25246
[1]	validation_0-rmse:0.23079
[2]	validation_0-rmse:0.21280
[3]	validation_0-rmse:0.19461
[4]	validation_0-rmse:0.17966
[5]	validation_0-rmse:0.16630
[6]	validation_0-rmse:0.15371
[7]	validation_0-rmse:0.14146
[8]	validation_0-rmse:0.13090
[9]	validation_0-rmse:0.12138
[10]	validation_0-rmse:0.11314
[11]	validation_0-rmse:0.10598
[12]	validation_0-rmse:0.09942
[13]	validation_0-rmse:0.09398
[14]	validation_0-rmse:0.08909
[15]	validation_0-rmse:0.08365
[16]	validation_0-rmse:0.07976
[17]	validation_0-rmse:0.07643
[18]	validat

C:\Users\HP\AppData\Local\Temp\ipykernel_9220\1753597180.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda':trail.suggest_loguniform('lambda',1e-4,10.0),
C:\Users\HP\AppData\Local\Temp\ipykernel_9220\1753597180.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha':trail.suggest_loguniform('alpha',1e-4,10.0),


[37]	validation_0-rmse:0.05610
[38]	validation_0-rmse:0.05593
[39]	validation_0-rmse:0.05620
[40]	validation_0-rmse:0.05608
[41]	validation_0-rmse:0.05620
[42]	validation_0-rmse:0.05686
[43]	validation_0-rmse:0.05683
[44]	validation_0-rmse:0.05722
[45]	validation_0-rmse:0.05731
[46]	validation_0-rmse:0.05696
[47]	validation_0-rmse:0.05720
[48]	validation_0-rmse:0.05730
[49]	validation_0-rmse:0.05715
[50]	validation_0-rmse:0.05704
[51]	validation_0-rmse:0.05728
[52]	validation_0-rmse:0.05730
[53]	validation_0-rmse:0.05739
[54]	validation_0-rmse:0.05768
[55]	validation_0-rmse:0.05798
[56]	validation_0-rmse:0.05797
[57]	validation_0-rmse:0.05840
[58]	validation_0-rmse:0.05820
[59]	validation_0-rmse:0.05813
[60]	validation_0-rmse:0.05803
[61]	validation_0-rmse:0.05807
[62]	validation_0-rmse:0.05818
[63]	validation_0-rmse:0.05831
[64]	validation_0-rmse:0.05799
[65]	validation_0-rmse:0.05823
[66]	validation_0-rmse:0.05801
[67]	validation_0-rmse:0.05799
[68]	validation_0-rmse:0.05788
[69]	val

[I 2022-12-31 19:32:31,956] Trial 0 finished with value: 0.0036475226500315983 and parameters: {'lambda': 2.7413245895602123, 'alpha': 0.07408891376218936, 'colsample_bytree': 0.8, 'subsample': 0.4, 'learning_rate': 0.1, 'max_depth': 12, 'random_state': 243123, 'min_child_weight': 71}. Best is trial 0 with value: 0.0036475226500315983.


[19:32:31] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "mim_child_weight", "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:5.07781
[1]	validation_0-rmse:80.84871
[2]	validation_0-rmse:1411.41388
[3]	validation_0-rmse:26751.34933
[4]	validation_0-rmse:508925.44969
[5]	validation_0-rmse:9408320.38255
[6]	validation_0-rmse:177053001.27246
[7]	validation_0-rmse:3350065992.09589
[8]	validation_0-rmse:63416060309.26608
[9]	validation_0-rmse:1200452994837.68823
[10]	validation_0-rmse:22725257612654.26172
[11]	validation_0-rmse:430210642899662.75000
[12]	validation_0-rmse:8145391411172531.00000
[13]	validation_0-rmse:154199981418192128.00000
[14]	validation_0-r

C:\Users\HP\AppData\Local\Temp\ipykernel_9220\1753597180.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda':trail.suggest_loguniform('lambda',1e-4,10.0),
C:\Users\HP\AppData\Local\Temp\ipykernel_9220\1753597180.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha':trail.suggest_loguniform('alpha',1e-4,10.0),
[W 2022-12-31 19:32:32,047] Trial 1 failed because of the following error: ValueError("could not convert string to float: '-nan(ind)'")
Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\HP\AppData\Lo

ValueError: could not convert string to float: '-nan(ind)'

In [ ]:
{'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [13]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.003648,2022-12-31 19:32:31.558644,2022-12-31 19:32:31.955829,0 days 00:00:00.397185,0.074089,0.8,2.741325,0.1,12,71,243123,0.4,COMPLETE
1,1,NaN,2022-12-31 19:32:31.956820,2022-12-31 19:32:32.047945,0 days 00:00:00.091125,0.001528,0.2,0.630561,20.0,8,195,3454,0.5,FAIL


In [14]:
optuna.visualization.plot_optimization_history(find_param)

In [15]:
optuna.visualization.plot_slice(find_param)

In [16]:
optuna.visualization.plot_contour(find_param , params = ['alpha' , 'lambda'])

[W 2022-12-31 19:33:42,191] Param alpha unique value length is less than 2.


In [17]:
best_param = {'lambda': 0.5646022891371127,
 'alpha': 0.35848071012137483,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 1,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 30}

In [18]:
xgb_final_model = xgb.XGBRegressor(**best_param)
xgb_final_model.fit(x,y)

XGBRegressor(alpha=0.35848071012137483, base_score=0.5, booster='gbtree',
             callbacks=None, colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', lambda=0.5646022891371127,
             learning_rate=1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=11, max_leaves=0, min_child_weight=30,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=30, ...)

In [19]:
xgb_final_model.score(x,y)

0.7342854588049831

In [33]:
#classification Task
df1 = pd.read_csv('D:\Python Programs\Machine Learning\winequality\winequality-red.csv')
df1 

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [34]:
df1.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [35]:
df1.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [37]:
X = df1.drop(columns='quality')
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [40]:
Y = df1['quality']
Y

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

In [47]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.20)
x_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1568,7.0,0.56,0.13,1.6,0.077,25.0,42.0,0.99629,3.34,0.59,9.2
720,8.4,0.56,0.04,2.0,0.082,10.0,22.0,0.99760,3.22,0.44,9.6
85,6.9,0.55,0.15,2.2,0.076,19.0,40.0,0.99610,3.41,0.59,10.1
386,7.8,0.54,0.26,2.0,0.088,23.0,48.0,0.99810,3.41,0.74,9.2
1517,6.5,0.53,0.06,2.0,0.063,29.0,44.0,0.99489,3.38,0.83,10.3
...,...,...,...,...,...,...,...,...,...,...,...
200,9.6,0.32,0.47,1.4,0.056,9.0,24.0,0.99695,3.22,0.82,10.3
1083,8.7,0.42,0.45,2.4,0.072,32.0,59.0,0.99617,3.33,0.77,12.0
609,6.3,0.36,0.19,3.2,0.075,15.0,39.0,0.99560,3.56,0.52,12.7
1431,7.6,0.43,0.31,2.1,0.069,13.0,74.0,0.99580,3.26,0.54,9.9


In [48]:
def objective_classification(trial):
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.20)
    param = {
        'tree_method' : 'hist',
        'verbosity':3,
        'objective':"binary:logistics",
        'booster' : trial.suggest_categorical('booster' , ['dart' , 'gbtree','gblinear']),
        'lambda' : trial.suggest_float('lambda' , 1e-4 , 1),
        'alpha' :trial.suggest_float('alpha' , 1e-4 , 1),
        'subsample' : trial.suggest_float('subsample' , .1,.5),
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , .1 ,.5)  
    }
    if param['booster'] in ['gbtree' , 'dart']:
        param['gamma'] :trial.suggest_float('gamma' , 1e-3 , 4 )
        param['eta'] : trial.suggest_float('eta' , .001 ,5 )
        
    xgb_classification = xgb.XGBClassifier(**param)
    xgb_classification.fit(x_train , y_train,eval_set  = [(x_test, y_test)])
    pred = xgb_classification.predict(x_test)
    accuracy  = xgb_classification.score(x_test , y_test)
    
    return accuracy


In [49]:
xgb_classification_oputuna = optuna.create_study(direction  = 'minimize')
xgb_classification_oputuna.optimize(objective_classification , n_trials = 10 )

[I 2022-12-31 20:04:57,032] A new study created in memory with name: no-name-c5f3e27e-6c77-49bc-a474-4b31c2b8520c
[W 2022-12-31 20:04:57,040] Trial 0 failed because of the following error: ValueError('Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]')
Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_9220\3941389337.py", line 18, in objective_classification
    xgb_classification.fit(x_train , y_train,eval_set  = [(x_test, y_test)])
  File "c:\Python310\lib\site-packages\xgboost\core.py", line 575, in inner_f
    return f(**kwargs)
  File "c:\Python310\lib\site-packages\xgboost\sklearn.py", line 1357, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]

In [55]:
best_param = {'booster': 'gblinear',
 'lambda': 0.33421885124456663,
 'alpha': 0.0055139194062208945,
 'subsample': 0.4357147003589471,
 'colsample_bytree': 0.47403249845643336}

In [58]:
xgb_final_clas = xgb.XGBClassifier(**best_param)
xgb_final_clas.fit(x_train , y_train)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]

In [59]:
xgb_final_clas.score(x_test , y_test)

NotFittedError: need to call fit or load_model beforehand